In [17]:
import requests
import os

import tensorflow as tf

SCORE_CAM_SOURCE = 'https://raw.githubusercontent.com/haofanwang/Score-CAM/master/images'
KERAS_VIS_SOURCE = 'https://raw.githubusercontent.com/keisen/tf-keras-vis/master/examples/images'

class config:
    class data:
        image_size = (512, 512)
        path = './data'
        images = os.path.join(path, 'misc')
        sources = [
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00000073.JPEG',
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00000091.JPEG',
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00000198.JPEG',
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00000476.JPEG',
            f'{SCORE_CAM_SOURCE}/ILSVRC2012_val_00002193.JPEG',

            f'{KERAS_VIS_SOURCE}/goldfish.jpg',
            f'{KERAS_VIS_SOURCE}/bear.jpg',
            f'{KERAS_VIS_SOURCE}/soldiers.jpg',

            'https://3.bp.blogspot.com/-W__wiaHUjwI/Vt3Grd8df0I/AAAAAAAAA78/7xqUNj8ujtY/s400/image02.png',
            'http://www.aviationexplorer.com/Diecast_Airplanes_Aircraft/delta_Airbus_diecast_airplane.jpg',
            'https://www.petcare.com.au/wp-content/uploads/2017/09/Dalmatian-2.jpg',
            'http://sites.psu.edu/siowfa15/wp-content/uploads/sites/29639/2015/10/dogcat.jpg',
            'https://consciouscat.net/wp-content/uploads/2009/08/multiple-cats-300x225.jpg',
            'https://images2.minutemediacdn.com/image/upload/c_crop,h_843,w_1500,x_0,y_78/f_auto,q_auto,w_1100/v1554995977/shape/mentalfloss/iStock-157312120.jpg',
            'http://www.reptilefact.com/wp-content/uploads/2016/08/Images-of-San-Francisco-Garter-Snake.jpg',
        ]

def load_images():
    global ds, class_names, file_names

    os.makedirs(config.data.images, exist_ok=True)

    file_names = [os.path.join(config.data.images, os.path.basename(f)) for f in config.data.sources]
    class_names = [os.path.splitext(os.path.basename(f))[0] for f in config.data.sources]
    
    for s, d in zip(config.data.sources, file_names):
        if not os.path.exists(d):
            download_image(s, d)
        
    ds = tf.data.Dataset.from_tensor_slices((file_names,))
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).cache()
    ds = ds.apply(tf.data.experimental.ignore_errors(log_warning=True))
    ds = ds.batch(len(file_names))

    return next(iter(ds))

def download_image(url, path):
    r = requests.get(url, allow_redirects=True)
    with open(path, 'wb') as f:
        f.write(r.content)
    return path

@tf.function
def load_image(x):
    x = tf.io.read_file(x)
    x = tf.image.decode_jpeg(x, channels=3)
    x = tf.image.resize(x, config.data.image_size)
    return x

@tf.function
def augment_fn(image, y):
    image = tf.image.random_brightness(image, .2)
    image = tf.image.random_contrast(image, .75, 1.)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    return image, y

x = load_images()

2022-06-11 20:30:44.207347: W tensorflow/core/kernels/data/experimental/ignore_errors_dataset_op.cc:117] Error raised with error message Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{function_node __inference_load_image_78331}}{{node DecodeJpeg}}]]
2022-06-11 20:30:44.207407: W tensorflow/core/kernels/data/experimental/ignore_errors_dataset_op.cc:117] Error raised with error message Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{function_node __inference_load_image_78331}}{{node DecodeJpeg}}]]
2022-06-11 20:30:44.207419: W tensorflow/core/kernels/data/experimental/ignore_errors_dataset_op.cc:117] Error raised with error message Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{function_node __inference_load_image_78331}}{{node DecodeJpeg}}]]
2022-06-11 20:30:44.216287: W tensorflow/core/kernels/data/experimental/ignore_errors_dataset_op.cc:117] Error raised with error message Unknown image file format. One of JPEG, PNG,

In [29]:
from keras.applications.imagenet_utils import preprocess_input

x_ = preprocess_input(x, mode='torch')

In [21]:
model = resnest.ResNeSt101(input_shape=(512, 512, 3), weights=None)
model.load_weights('weights/resnest101.h5')

In [33]:
tf.reduce_min(x), tf.reduce_min(xp)

(<tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-2.117904>)

In [34]:
tf.reduce_max(x), tf.reduce_max(xp)

(<tf.Tensor: shape=(), dtype=float32, numpy=255.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.64>)

In [41]:
outputs = model.predict(x_)

1/1 [==============================] - 14s 14s/step


In [40]:
predictions = tf.keras.applications.imagenet_utils.decode_predictions(outputs, top=3)
print(
    *(
        ' | '.join(f'{name.rjust(15)} {conf:.2%}' for _id, name, conf in p)
        for p in predictions
    ),
    sep='\n'
)

35363/35363 [==============================] - 0s 0us/step
           kite 69.49% | prairie_chicken 0.17% | sulphur-crested_cockatoo 0.15%
     white_wolf 83.62% |     timber_wolf 1.96% |  Great_Pyrenees 0.39%
         bulbul 75.84% |         jacamar 2.16% |    black_grouse 0.78%
   black_grouse 33.74% |     black_stork 27.77% |          magpie 4.78%
        obelisk 37.41% |        pedestal 36.72% |          beacon 2.55%
golden_retriever 73.72% |  Great_Pyrenees 6.12% |          kuvasz 1.43%
       airliner 80.57% |            wing 2.95% |   space_shuttle 0.31%
      dalmatian 74.94% |          kuvasz 0.27% |      Great_Dane 0.22%
          tabby 36.82% |       tiger_cat 11.71% |             tub 10.33%
     spider_web 54.00% |     barn_spider 10.14% |   garden_spider 8.33%
   garter_snake 69.39% |   thunder_snake 2.47% |      sidewinder 1.89%
